In [1]:
import numpy as np
from random import randint
from math import floor, fabs
from cplex.callbacks import UserCutCallback
import cplex  as CPX
import cplex.callbacks as CPX_CB
import sys

def SecModelParameter ( VarsValue, m, n, my_rhs, my_coef, BranchedVariables):
#     print 'BranchedVariables',BranchedVariables
    unbranched=list()
    c=[]
    d=(np.zeros(((2 * m) + (4 * n), 2*n)))
    halatest=[]
    a=[]
    newrow=[]
    zer=(np.zeros(((2 * m) + (4 * n), 2*n)))
    for i in range (n):
        if i not in (BranchedVariables):
            unbranched.append(i)
    NodeNum=n-len(unbranched)
    
    
    for i in unbranched:   
        for h in range (m):
            c.append(0)
        for h in range (m):
            lin=0            
            for j in range (n):
#                 print 'i', i, 'h', h,'j',j
#                 print 'j1',j
                if j in (BranchedVariables):
#                     print 'j2',j
                    sv = VarsValue[BranchedVariables.index(j)]
#                     print 'sv', sv
                    lin =lin+ (sv * my_coef[h][j]) 
#                     print 'lin', lin
#                 print 'sv'
            lin=lin- my_rhs[h]
            c.append(lin)                
        for h in range (n):
            c.append(0)
        for h in range (n):
            c.append(0)

        for jj in range (n):
            lin2=-1
            if jj in (BranchedVariables):
                sv = VarsValue[BranchedVariables.index(jj)]
                lin2 =lin2+ sv
            c.append(lin2)
            
        for jj in range (n):
            lin2=0
            if jj in (BranchedVariables):
                lin2 =-1*VarsValue[BranchedVariables.index(jj)]
            c.append(lin2)

              
    k=NodeNum-1 

    for i in unbranched: 
        k=k+1
        for t in range(m):              
            for j in range(n):    
                d[(k-NodeNum)*(2*m+4*n)+t][j+n]=my_coef[t][j] 
                
        for t in range(m):  
            d[(k-NodeNum)*(2*m+4*n)+t][n+i]=d[(k-NodeNum)*(2*m+4*n)+t][n+i]-my_rhs[t]   
                
        for t in range(m):  
            for kk in unbranched:
                d[(k-NodeNum)*(2*m+4*n)+t+m][kk]=my_coef[t][kk] 
                

        for t in range(m):  
            for j in range(n):
                d[(k-NodeNum)*(2*m+4*n)+t+m][j+n]=-1*my_coef[t][j] 
                 
        for t in range(m):  
            d[(k-NodeNum)*(2*m+4*n)+t+m][n+i]=d[(k-NodeNum)*(2*m+4*n)+t+m][n+i]+my_rhs[t] 
                
        for t in range(n):              
            d[(k-NodeNum)*(2*m+4*n)+t+2*m][i]=-1 
            
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+t][n+t]=1 
        
            
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+n+t][n+t]=-1 
        
        for mm in unbranched: 
             
            d[(k-NodeNum)*(2*m+4*n)+2*m+2*n+mm][mm]=1  
            
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+t+2*n+2*m][i]=d[(k-NodeNum)*(2*m+4*n)+t+2*n+2*m][i]+1            
            

        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+2*n+t][n+t]=-1 

        for t in unbranched:  
            d[(k-NodeNum)*(2*m+4*n)+2*m+3*n+t][t]=-1  
               
        
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+3*n+t][n+t]=1 
            
        d=np.vstack([d, zer])                          
            
    d = np.delete(d, slice(((n - NodeNum)*((2 * m) + (4 * n))),((n-NodeNum+1)*((2 * m) + (4 * n)))), axis=0)
#     print 'c',c
#     print 'unbranched',unbranched
    return c, d,unbranched
#     print c, d,unbranched

In [2]:
def SecondModel (cc,dd,NotBranched):
#     print 'cc,dd,NotBranched',cc,dd,NotBranched
    numrows = len(dd)    # 3 rows in your example
    numcols = len(dd[0])
    VarNum=numcols/2
    ConstNum=numrows/len(NotBranched)
    my_obj=[]
    my_ub=[]
    my_lb=[]
    my_colnames=[]
    my_rownames=["c1"]
    my_coef=[] 
    my_row=[]
    my_rhs=[]
    my_sense=""
    my_Nmah=list()
    
    for i in range(numrows):
        my_obj=cc
        my_ub.append(1)
        my_lb.append(0)
        my_colnames.append("y"+str(i))

    sumarray=[]     
    NotBranched.sort()    
#     sumshode=list()
    for t in range (VarNum):
        sumlist=list()
        for j in range(numrows):
#             print 't,j',t,j
            if t in NotBranched: 
                indj=NotBranched.index(t)
#                 print 'indj',indj
                indjrange=range(indj*ConstNum,(indj+1)*ConstNum)
#                 print 'indjrange',indjrange
                if j in indjrange:
#                     print 'dd[j,t]+dd[j,t+VarNum]',j,'ll',dd[j,t]+dd[j,t+VarNum]
                    sumlist.append(dd[j,t]+dd[j,t+VarNum])
                    dd[j,t+VarNum]=0
                else:
                    sumlist.append(dd[j,t])
            else:
                sumlist.append(dd[j,t])
        sumarray.append(sumlist)
#     print 'sumarray',sumarray
#     print 'newDD',dd
  
#     print 'notbranched',NotBranched
    lenNotbranched=len(NotBranched)
   
    
    for t in range(lenNotbranched):  
        for i in range (VarNum):
            sumlist=list()
            if t>0:
                for p in range(t):
                    for j in range (ConstNum):
                        sumlist.append(0) 
                        
            for j in range(ConstNum):
                sumlist.append(dd[t*ConstNum+j,i+VarNum])
            if i in NotBranched and i > NotBranched[t]:
                indx=NotBranched.index(i)
                indy=NotBranched.index(NotBranched[t])
                
                for x in range(indx-indy-1):
                    for j in range (ConstNum):
                        sumlist.append(0) 
                for j in range(ConstNum):
                    sumlist.append(dd[(indx)*ConstNum+j,NotBranched[t]+VarNum])
                    dd[(indx)*ConstNum+j,NotBranched[t]+VarNum]=0
#                 sumarray.append([t+indx,indy])
                if t<lenNotbranched-2 :
                    for p in range(lenNotbranched-indx-1):
                        for j in range (ConstNum):
                            sumlist.append(0)
            else:
                if t<lenNotbranched-1 :
#                     print 'lenNotbranched',lenNotbranched
                    for p in range(lenNotbranched-t-1):
                            for j in range (ConstNum):
                                sumlist.append(0) 
#             print 't,i',t,',',i
#             print 'sumlist p2',sumlist
#             print 'lensumlist p2',len(sumlist)
            sumarray.append(sumlist)
#     print 'part2' ,sumarray  
    
    rows=[]
    for j in range(numrows):
        my_row.append(1)
    rows.append([range(numrows),my_row])  
    def populatebyrow(SecProb):
        SecProb.objective.set_sense(SecProb.objective.sense.maximize)
        SecProb.variables.add(obj = my_obj,lb=my_lb, names = my_colnames)
        
        for i in range (len(sumarray)):
            if all(k == 0 for k in sumarray[i])!=True:
#                 print 'i',i
                sumline=[]
                sumline.append([range(len(sumarray[i])),sumarray[i]])
#                 print 'sumline',sumline
                SecProb.linear_constraints.add(lin_expr = sumline, senses =  "E",
                                rhs = [0], names = ["u"+str(i)])
        SecProb.linear_constraints.add(lin_expr = rows, senses = "L",
                                             rhs = [1], names = ["t0"])
    
    my_SecProb = CPX.Cplex()
    handle = populatebyrow(my_SecProb)
    my_SecProb.solve()
    my_SecProb.write("lpex2.lp")
    SecVariables    = my_SecProb.solution.get_values()
    SecObjective=my_SecProb.solution.get_objective_value()
#     print 'SecVariables,SecObjective,my_colnames',SecVariables, SecObjective,my_colnames
    return SecVariables, SecObjective,my_colnames

In [3]:
def GeneratintCutParameters ( m, n, my_rhs, my_coef, BranchedVariables,SecVariables): 
    BranchedLen=len(BranchedVariables)
    unbranched=list()
    for i in range (n):
        if i not in (BranchedVariables):
            unbranched.append(i)  
    LeUnbranched=len(unbranched)                   
#     NodeNum=n-len(unbranched)
#     k=NodeNum-1 
    lhs=(np.zeros((LeUnbranched*((2 * m) + (4 * n)),BranchedLen)))
    rhs=(np.zeros((LeUnbranched*((2 * m) + (4 * n)),1)))

    for i in range (LeUnbranched):   
        for h in range (m):
            for j in (BranchedVariables):
                lhs[i*(2*m+4*n)+m+h][BranchedVariables.index(j)]=my_coef[h][j]*SecVariables[i*(2*m+4*n)+m+h]  
                rhs[i*(2*m+4*n)+m+h]=my_rhs[h]*SecVariables[i*(2*m+4*n)+m+h]  
            
        for h in range (n):
            for j in (BranchedVariables):
                lhs[i*(2*m+4*n)+2*m+2*n+j][BranchedVariables.index(j)]=SecVariables[i*(2*m+4*n)+2*m+2*n+j]  
                rhs[i*(2*m+4*n)+2*m+2*n+h]=SecVariables[i*(2*m+4*n)+2*m+2*n+h]  
            
#         for h in range (n):
        for j in (BranchedVariables):
            lhs[i*(2*m+4*n)+2*m+3*n+j][BranchedVariables.index(j)]=-1*SecVariables[i*(2*m+4*n)+2*m+3*n+j]  

    #     print 'lefttt',lhs,'rightttt', rhs
    LenLeft=len(lhs)
    LenRight=len(rhs)
    LenSec=len(SecVariables)
#     print 'LenLeft',LenLeft
#     print'LenRight',LenRight
#     print 'LenSec',LenSec
#     indicesLhs=[]
#     for j in range (8):
#         indicesLhs.append([i for i in range(LenLeft) if lhs[i][j]!=0]) 
#     print indicesLhs
#     indicesRhs = [(i,rhs[i]) for i in range(LenRight) if rhs[i]>0]
#     print 'indicesRhs',indicesRhs
#     indiceSecVariables=[(i , SecVariables[i]) for i in range(LenSec) if SecVariables[i]!=0]
#     ValueSecVariables=[SecVariables[i] for i in range(LenSec) if SecVariables[i]!=0]
#     print 'positiveLeftHand',indicesLhs 
#     print 'indiceSecVariables',indiceSecVariables
#     print 'ValueSecVariables', ValueSecVariables
    FinalLhs=np.sum(lhs, axis=0)
    FinalRhs=np.sum(rhs, axis=0)
    print 'FinalLhs',FinalLhs, 'FinalRhs',FinalRhs
    return FinalLhs,FinalRhs



In [4]:
class MyCut(CPX_CB.UserCutCallback):

    def __call__(self):
         
        self.times_called += 1
        print  'self.times_called', self.times_called
        branched=list()
        values=list()
        branchedNames=list()
        global BranchedMatrix
        senses= "L"
#         print 'self.get_lower_bounds(s)',self.get_lower_bounds()
#         print 'self.get_upper_bounds(s)',self.get_upper_bounds()
#         print 'objective value', self.get_objective_value()
#         print 'values', self.get_values()
        print 'get_num_nodes', self.get_num_nodes()

        lb=self.get_lower_bounds()
        ub=self.get_upper_bounds()               
        for j in range(len(lb)):
            if lb[j]==ub[j]:
                branched.append(j)
                values.append(lb[j])
                branchedNames.append(VarsName[j])
                
        print 'values',values
        print 'branchedNames',branchedNames
        print 'branched',branched

#         if len(values)>0:  
#             BranchedMatrix.append([branched,values])
# #             BranchedValue.append(values)
# #             print 'BranchedMatrix',BranchedMatrix

#             if [BranchedMatrix[-1]]!=[BranchedMatrix[-2]] :
#                 SecModelParameterOutput=SecModelParameter(values, self.m, self.n, self.my_rhs, self.my_coef,branched)
#                 SecondModelOutput=SecondModel (SecModelParameterOutput[0],SecModelParameterOutput[1],SecModelParameterOutput[2])
#                 print 'SecondModelOutput[1]',SecondModelOutput[1]
#                 print 'SecondModelOutput[0]',SecondModelOutput[0]
#                 if SecondModelOutput[1]>0.000001:
#     #                 print 'bayad cut bezane'
#                     output=GeneratintCutParameters (self.m, self.n, self.my_rhs, self.my_coef,branched, SecondModelOutput[0])
#                     print 'output',output[0]
#                     print 'output[1]',output[1]
#                     lhs=[CPX.SparsePair(ind =branchedNames, val = output[0])]
#                     print 'lhs',lhs[0]
#                     self.add(cut=lhs[0],rhs=output[1][0],sense=senses)
#                     print 'cut added'



In [5]:
def admipex3(c):
    
    c.parameters.preprocessing.presolve = 0

    c.parameters.preprocessing.reduce = 0

    c.parameters.preprocessing.linear.set(0)

    c.parameters.preprocessing.numpass = 0

    c.parameters.mip.strategy.presolvenode = -1

    c.parameters.preprocessing.linear = 0
    
def CutCplex(c):
    c.parameters.mip.cuts.disjunctive.set(-1)
    c.parameters.mip.cuts.bqp.set(-1)
    c.parameters.mip.cuts.cliques.set(-1)
    c.parameters.mip.cuts.covers.set(-1)
    c.parameters.mip.cuts.flowcovers.set(-1)
    c.parameters.mip.cuts.gomory.set(-1)
    c.parameters.mip.cuts.gubcovers.set(-1)
    c.parameters.mip.cuts.implied.set(-1)
    c.parameters.mip.cuts.liftproj.set(-1)
    c.parameters.mip.cuts.localimplied.set(-1)
    c.parameters.mip.cuts.mcfcut.set(-1)
    c.parameters.mip.cuts.mircut.set(-1)
    c.parameters.mip.cuts.zerohalfcut.set(-1)
    c.parameters.mip.cuts.pathcut.set(-1)
    
c = CPX.Cplex('NewExample.lp')
m=c.linear_constraints.get_num()
n=c.variables.get_num()
my_rhs=[]
my_coef=[]
for i in range(m):
    my_row=[]
    for j in range(n):
        my_row.append(c.linear_constraints.get_coefficients(i,j))
    my_coef.append(my_row)

for i in range (m):
    my_rhs.append(c.linear_constraints.get_rhs(i))
BranchOrdering=[]
for i in range (n):
    BranchOrdering.append((i,i+1,c.order.branch_direction.default))
c.order.set(BranchOrdering)
print 'order',c.order.get()

admipex3(c)
CutCplex(c)
VarsName=c.variables.get_names()
c.set_log_stream(sys.stdout)
c.set_results_stream(sys.stdout)

cutinst=c.register_callback(MyCut)
cutinst.times_called = 0
cutinst.m=m
cutinst.n=n
cutinst.my_rhs=my_rhs
cutinst.my_coef=my_coef
cutinst.VarsName=VarsName
BranchedMatrix=[[[-1], [-0.0]]]
c.parameters.mip.interval.set(1)
c.parameters.preprocessing.linear=0
c.parameters.mip.strategy.search.set(c.parameters.mip.strategy.search.values.traditional)
c.solve()

c.write("lpex1.lp")
solution = c.solution

# the following line prints the corresponding string
print solution.status[solution.get_status()]
print "Objective value = " , solution.get_objective_value()
print
x = solution.get_values(0, c.variables.get_num()-1)
for j in range(c.variables.get_num()):
    if fabs(x[j]) > 1.0e-10:
        print "Column %d: Value = %17.10g" % (j, x[j])

print "Branch callback was called ", cutinst.times_called, "times"


order [(0, 1, 0), (1, 2, 0), (2, 3, 0), (3, 4, 0), (4, 5, 0), (5, 6, 0), (6, 7, 0), (7, 8, 0), (8, 9, 0), (9, 10, 0), (10, 11, 0), (11, 12, 0), (12, 13, 0), (13, 14, 0), (14, 15, 0), (15, 16, 0), (16, 17, 0), (17, 18, 0), (18, 19, 0), (19, 20, 0), (20, 21, 0), (21, 22, 0), (22, 23, 0), (23, 24, 0), (24, 25, 0), (25, 26, 0), (26, 27, 0), (27, 28, 0), (28, 29, 0), (29, 30, 0)]
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Preprocessing_Linear                    0
CPXPARAM_MIP_Cuts_Cliques                        -1
CPXPARAM_MIP_Cuts_Covers                         -1
CPXPARAM_MIP_Interval                            1
CPXPARAM_MIP_Cuts_FlowCovers                     -1
CPXPARAM_MIP_Cuts_Implied                        -1
CPXPARAM_MIP_Cuts_GUBCovers                      -1
CPXPARAM_MIP_Cuts_Gomory                         -1
CPXPARAM_MIP_Cuts_PathCut                        -1
CPXPARAM_MIP_Cuts_MIRCut                         -1
CPXPARAM_MIP_Cuts_Disjunctive                    -1
C

     12    14     1352.6772     6     1171.0000     1422.8501       46   21.51%             x14 U     12     11     11
self.times_called 27
get_num_nodes 13
values [-0.0, 1.0, 1.0, -0.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
branchedNames ['x9', 'x12', 'x14', 'x16', 'x18', 'x19', 'x21', 'x22', 'x24', 'x25', 'x26', 'x27', 'x28']
branched [9, 12, 14, 16, 18, 19, 21, 22, 24, 25, 26, 27, 28]
self.times_called 28
get_num_nodes 13
values [-0.0, 1.0, 1.0, -0.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
branchedNames ['x9', 'x12', 'x14', 'x16', 'x18', 'x19', 'x21', 'x22', 'x24', 'x25', 'x26', 'x27', 'x28']
branched [9, 12, 14, 16, 18, 19, 21, 22, 24, 25, 26, 27, 28]
     13    15     1352.2615     5     1171.0000     1422.8501       48   21.51%              x9 D     13     12     12
self.times_called 29
get_num_nodes 14
values [1.0, -0.0, 1.0, 1.0, -0.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
branchedNames ['x6', 'x9', 'x12', 'x14', 'x16', 'x18', 'x19', 'x21', 'x22', 'x24

     26    21     1403.6034     3     1171.0000     1418.7155       73   21.15%              x9 D     26     25      6
self.times_called 53
get_num_nodes 27
values [1.0, -0.0, 1.0, -0.0, 1.0, 1.0, -0.0]
branchedNames ['x8', 'x9', 'x16', 'x19', 'x22', 'x24', 'x27']
branched [8, 9, 16, 19, 22, 24, 27]
self.times_called 54
get_num_nodes 27
values [1.0, -0.0, 1.0, -0.0, 1.0, 1.0, -0.0]
branchedNames ['x8', 'x9', 'x16', 'x19', 'x22', 'x24', 'x27']
branched [8, 9, 16, 19, 22, 24, 27]
     27    22     1403.3816     3     1171.0000     1418.7155       74   21.15%              x8 U     27     26      7
self.times_called 55
get_num_nodes 28
values [1.0, -0.0, -0.0, 1.0, -0.0, 1.0, 1.0, -0.0]
branchedNames ['x8', 'x9', 'x10', 'x16', 'x19', 'x22', 'x24', 'x27']
branched [8, 9, 10, 16, 19, 22, 24, 27]
self.times_called 56
get_num_nodes 28
values [1.0, -0.0, -0.0, 1.0, -0.0, 1.0, 1.0, -0.0]
branchedNames ['x8', 'x9', 'x10', 'x16', 'x19', 'x22', 'x24', 'x27']
branched [8, 9, 10, 16, 19, 22, 24, 27]


     38    31     1382.7210     4     1224.0000     1418.7155       94   15.91%             x23 U     38     37     18
self.times_called 77
get_num_nodes 39
values [1.0, 1.0, -0.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0, -0.0, 1.0, -0.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, 1.0]
branchedNames ['x1', 'x3', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x14', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28']
branched [1, 3, 5, 6, 7, 8, 9, 10, 11, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]
self.times_called 78
get_num_nodes 39
values [1.0, 1.0, -0.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0, -0.0, 1.0, -0.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, 1.0]
branchedNames ['x1', 'x3', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x14', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28']
branched [1, 3, 5, 6, 7, 8, 9, 10, 11, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]
     39    32    

     54    40     1367.6562     7     1224.0000     1417.5658      129   15.81%             x24 D     54      2      2
self.times_called 103
get_num_nodes 55
values [1.0, -0.0, 1.0, 1.0]
branchedNames ['x3', 'x24', 'x26', 'x27']
branched [3, 24, 26, 27]
self.times_called 104
get_num_nodes 55
values [1.0, -0.0, 1.0, 1.0]
branchedNames ['x3', 'x24', 'x26', 'x27']
branched [3, 24, 26, 27]
     55    41     1340.6694     7     1224.0000     1417.5658      134   15.81%             x26 U     55     54      3
self.times_called 105
get_num_nodes 56
values [1.0, 1.0, -0.0, 1.0, 1.0, -0.0, 1.0, 1.0]
branchedNames ['x2', 'x3', 'x17', 'x20', 'x22', 'x24', 'x26', 'x27']
branched [2, 3, 17, 20, 22, 24, 26, 27]
self.times_called 106
get_num_nodes 56
values [1.0, 1.0, -0.0, 1.0, 1.0, -0.0, 1.0, 1.0]
branchedNames ['x2', 'x3', 'x17', 'x20', 'x22', 'x24', 'x26', 'x27']
branched [2, 3, 17, 20, 22, 24, 26, 27]
     56    42     1334.6606     8     1224.0000     1417.5658      137   15.81%             x22 

...

    303   135    infeasible           1268.0000     1354.5692      716    6.83%              x7 D    303    301     11
self.times_called 483
get_num_nodes 304
values [1.0, -0.0, 1.0, 1.0, -0.0, -0.0, 1.0, -0.0, 1.0, -0.0, -0.0, 1.0, 1.0, -0.0]
branchedNames ['x3', 'x5', 'x7', 'x8', 'x9', 'x10', 'x11', 'x14', 'x16', 'x19', 'x21', 'x22', 'x24', 'x27']
branched [3, 5, 7, 8, 9, 10, 11, 14, 16, 19, 21, 22, 24, 27]
self.times_called 484
get_num_nodes 304
values [1.0, -0.0, 1.0, 1.0, -0.0, -0.0, 1.0, -0.0, 1.0, -0.0, -0.0, 1.0, 1.0, -0.0]
branchedNames ['x3', 'x5', 'x7', 'x8', 'x9', 'x10', 'x11', 'x14', 'x16', 'x19', 'x21', 'x22', 'x24', 'x27']
branched [3, 5, 7, 8, 9, 10, 11, 14, 16, 19, 21, 22, 24, 27]
    304   135     1282.2803     5     1268.0000     1354.5692      724    6.83%             x14 D    304     31     12
    305   134    infeasible           1268.0000     1354.5692      724    6.83%             x26 U    305    304     13
self.times_called 485
get_num_nodes 306
values [1.0, -0

    318   138     1350.0715     6     1268.0000     1352.7910      752    6.69%             x15 U    318    317      7
self.times_called 503
get_num_nodes 319
values [1.0, 1.0, -0.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
branchedNames ['x2', 'x3', 'x5', 'x11', 'x15', 'x16', 'x17', 'x18', 'x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'x27', 'x28', 'x29']
branched [2, 3, 5, 11, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 27, 28, 29]
self.times_called 504
get_num_nodes 319
values [1.0, 1.0, -0.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
branchedNames ['x2', 'x3', 'x5', 'x11', 'x15', 'x16', 'x17', 'x18', 'x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'x27', 'x28', 'x29']
branched [2, 3, 5, 11, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 27, 28, 29]
    319   139     1349.6657     5     1268.0000     1352.7910      753    6.69%             x25 U    319    318      8
self.times_called 505
get_num_nodes 320
values [1.0, 1.0, -0.0, 1.0, 1.0, 

    334    66     1333.3682     9     1317.0000     1347.2534      789    2.30%             x16 D    334    261      5
self.times_called 521
get_num_nodes 335
values [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0, -0.0, 1.0, -0.0, 1.0, 1.0, -0.0, 1.0, -0.0, 1.0, 1.0, 1.0]
branchedNames ['x1', 'x2', 'x3', 'x7', 'x8', 'x11', 'x12', 'x13', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28', 'x29']
branched [1, 2, 3, 7, 8, 11, 12, 13, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
self.times_called 522
get_num_nodes 335
values [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0, -0.0, 1.0, -0.0, 1.0, 1.0, -0.0, 1.0, -0.0, 1.0, 1.0, 1.0]
branchedNames ['x1', 'x2', 'x3', 'x7', 'x8', 'x11', 'x12', 'x13', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28', 'x29']
branched [1, 2, 3, 7, 8, 11, 12, 13, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
    335    66     1324.0184     7     1

    361    45        cutoff           1317.0000     1337.9767      863    1.59%             x13 D    361     47      8
self.times_called 535
get_num_nodes 362
values [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, -0.0, -0.0, 1.0, 1.0, 1.0, -0.0, 1.0, -0.0, -0.0, 1.0, 1.0]
branchedNames ['x1', 'x2', 'x3', 'x7', 'x11', 'x15', 'x16', 'x17', 'x19', 'x20', 'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28', 'x29']
branched [1, 2, 3, 7, 11, 15, 16, 17, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29]
self.times_called 536
get_num_nodes 362
values [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, -0.0, -0.0, 1.0, 1.0, 1.0, -0.0, 1.0, -0.0, -0.0, 1.0, 1.0]
branchedNames ['x1', 'x2', 'x3', 'x7', 'x11', 'x15', 'x16', 'x17', 'x19', 'x20', 'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28', 'x29']
branched [1, 2, 3, 7, 11, 15, 16, 17, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29]
    362    46     1328.0423     6     1317.0000     1337.0324      867    1.52%             x16 D    362    323      6
self.times_called 537
get_num_nodes 363
valu

    386    30     1322.5927     5     1317.0000     1328.2865      929    0.86%             x18 U    386    385      8
    387    29        cutoff           1317.0000     1328.2865      931    0.86%             x13 U    387    386      9
    388    28        cutoff           1317.0000     1328.0130      934    0.84%             x18 D    388    207      7
    389    27    infeasible           1317.0000     1327.8957      934    0.83%              x5 U    389    141     14
    390    26        cutoff           1317.0000     1327.6407      936    0.81%             x10 U    390     95      7
Elapsed time = 4.92 sec. (20.60 ticks, tree = 0.01 MB, solutions = 8)
    391    25        cutoff           1317.0000     1327.1954      938    0.77%             x21 U    391    148     10
    392    24        cutoff           1317.0000     1326.9763      942    0.76%             x21 U    392     56      5
self.times_called 551
get_num_nodes 393
values [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, -0.0, -0